In [1]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import nibabel as nib
from tensorflow import keras
from tensorflow.keras import layers
from scipy import ndimage

In [2]:
def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan


def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume


def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img


def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(path)
    # Normalize
    volume = normalize(volume)
    # Resize width, height and depth
    volume = resize_volume(volume)
    return volume

In [3]:
scan_paths = [
    os.path.join(os.getcwd(), "../BET_BSE_DATA", x)
    for x in os.listdir("../BET_BSE_DATA")
]

print("Brain scans: " + str(len(scan_paths)))

Brain scans: 4


In [4]:
import csv
 
# csv file name
filename = "/home/joelkik/DataMining/BET_BSE_DATA/Label_file.csv"
 
# initializing the titles and rows list
fields = []
rfc = set()
nrfc = set()
 
# reading csv file
with open(filename, 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
     
    # extracting field names through first row
    fields = next(csvreader)
 
    # extracting each data row one by one
    for row in csvreader:
        #print(row)
        if row[1].lower() == 'yes':
            rfc.add(row[0]+'.gz')
        else:
            nrfc.add(row[0]+'.gz')
 
    # get total number of rows
    print("Total no. of rows: %d"%(csvreader.line_num))
 
# printing the field names
print('Field names are:' + ', '.join(field for field in fields))
#print(labels_dict)

Total no. of rows: 1312
Field names are:Filename, Recognizable-Facial-Feature, Brain-Feature-Loss


In [5]:
import random

from scipy import ndimage


@tf.function
def rotate(volume):
    """Rotate the volume by a few degrees"""

    def scipy_rotate(volume):
        # define some rotation angles
        angles = [-20, -10, -5, 5, 10, 20]
        # pick angles at random
        angle = random.choice(angles)
        # rotate volume
        volume = ndimage.rotate(volume, angle, reshape=False)
        volume[volume < 0] = 0
        volume[volume > 1] = 1
        return volume

    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)
    return augmented_volume


def train_preprocessing(volume, label):
    """Process training data by rotating and adding a channel."""
    # Rotate volume
    volume = rotate(volume)
    volume = tf.expand_dims(volume, axis=3)
    return volume, label


def validation_preprocessing(volume, label):
    """Process validation data by only adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label

In [6]:
# Read and process the scans.
# Each scan is resized across height, width, and depth and rescaled.
rfc_scans=[]
nrfc_scans=[]
labels_facial_feature=[]
labels_to_file=[]
for path in os.listdir("/home/joelkik/DataMining/BET_BSE_DATA/files"):
    if path in rfc:
        rfc_scans.append(process_scan("/home/joelkik/DataMining/BET_BSE_DATA/files/"+path))
    else:
        nrfc_scans.append(process_scan("/home/joelkik/DataMining/BET_BSE_DATA/files/"+path))
        print(path)
    if  len(nrfc_scans)>2:
        break
    #labels_facial_feature.append(labels_dict[path])
    #labels_to_file.append(path)

IXI115-Guys-0738-T1_bse_high_s76_r2.nii.gz
IXI218-HH-1815-T1_bse_high_s74_r2.nii.gz
IXI038-Guys-0729-T1_bet_85.nii.gz


In [7]:
model = keras.models.load_model("/home/joelkik/DataMining/CPSC8650_Project/3d_reconstruct_facial_features.h5")

In [8]:
for i in range(len(nrfc_scans)):
    #print(np.expand_dims(nrfc_scans[i], axis=0).shape)
    prediction = model.predict(np.expand_dims(nrfc_scans[i], axis=0))[0][:,:,:,0]
    #print(prediction.shape)
    ni_img = nib.Nifti1Image(prediction, affine=np.eye(4))
    #print(ni_img.shape)
    nib.save(ni_img, os.path.join('ReconstructedImages', 'test_{}.nii.gz'.format(i)))
    #nib.save(os.path.join('ReconstructedImages', 'test_{}.nii.gz'.format(i)), prediction) 
    #print("Scan File: ", labels_to_file[i], ", Actual: ", labels_facial_feature_np[i], ", Predicted: ",round(float(prediction)))